In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tushare as ts
import warnings; warnings.simplefilter('ignore') #忽略可能会出现的警告信息，警告并不是错误，可以忽略；

#获取飞机制造行业相关股票代码
industry_classified = ts.get_industry_classified()
aero_industry = industry_classified[industry_classified['c_name'].isin(['飞机制造'])]
aero_industry_code = aero_industry['code'].tolist()

#获取相关股票基本面数据
stock_basics = ts.get_stock_basics()
stock_basics.reset_index(inplace=True)      #reset_index重置dataFrame的索引
data = stock_basics.loc[stock_basics['code'].isin(aero_industry_code),
                         ['code', 'name', 'industry', 'pe', 'pb', 'esp','rev', 'profit',]]
data.columns = ['代码','名称','行业','PE','PB','EPS','收入%','利润%']

# 获取相关公司2018年第一季度盈利能力数据
stock_profit = ts.get_profit_data(2018,1)
data1 = stock_profit.loc[stock_profit['code'].isin(aero_industry_code), ['code', 'roe', 'gross_profit_rate', 'net_profit_ratio']]
data1.columns = ['代码', 'ROE', '毛利率', '净利率']
data1 = round(data1,2)

# 获取成长能力数据
stock_growth = ts.get_growth_data(2018,1)
data2 = stock_growth.loc[stock_growth['code'].isin(aero_industry_code), ['code', 'nprg']]  # nprg:净利润增长率（%）
data2.columns = ['代码', 'NI%', ]
data2 = round(data2,2)
data2

#合并表格
data = pd.merge(data ,data1, how='left', on='代码')
data = pd.merge(data ,data2, how='left', on='代码')
data.drop_duplicates(inplace = True)        #去除重复数据

# 根据已有列计算新数据
data['估值系数'] = data['PE'] * data['PB']
data = round(data,2)

# 数据分类
def map_func(x):
    
    if x['ROE'] > 5:
        return '高成长'
    elif x['ROE'] >= 0:
        return '低成长'
    elif x['ROE'] < 0:
        return '亏损'

# 根据 ROE 数据计算“成长性”
data['成长性'] = data.apply(map_func, axis=1)
data

[Getting data:]###########################################################[Getting data:]###########################################################

,代码,名称,行业,PE,PB,EPS,收入%,利润%,ROE,毛利率,净利率,NI%,估值系数,成长性
0,002023,海特高新,航空,63.52,2.61,0.05,0.96,229.91,1.02,40.84,34.75,475.94,165.79,低成长
1,600118,中国卫星,航空,89.96,4.12,0.05,13.63,3.27,1.14,14.63,4.71,10.24,370.64,低成长
2,002111,威海广泰,航空,20.03,1.54,0.13,15.45,24.30,1.91,35.96,13.26,22.13,30.85,低成长
3,000738,航发控制,航空,67.64,3.03,0.05,-2.62,58.93,1.11,29.49,10.55,61.33,204.95,低成长
4,600893,航发动力,航空,0.00,1.97,-0.03,1.80,36.03,-0.22,18.08,-2.07,NaN,0.00,亏损
5,300424,航新科技,航空,638.20,5.56,0.01,-4.52,7.44,0.21,43.86,2.59,0.04,3548.39,低成长
6,600879,航天电子,航空,44.39,1.54,0.04,9.53,10.17,0.86,16.17,3.73,14.39,68.36,低成长
7,600391,航发科技,航空,0.00,2.61,-0.04,-3.38,-465.28,-0.65,20.53,-3.03,NaN,0.00,亏损
9,600316,洪都航空,航空,0.00,1.32,-0.03,-52.33,27.31,-0.42,6.73,-10.41,NaN,0.00,亏损
10,000901,航天科技,航空,0.00,1.62,-0.02,-0.41,-148.87,-0.27,19.13,-0.75,-138.97,0.00,亏损
